# Chemical kinetics for mineral reactions using Palandri-Kharaka model

<p class="acknowledgement">Written by Allan Leal (ETH Zurich) on Nov 21th, 2022</p>

In the previous tutorial, we learned how to define our own reaction rate model to calculate the dissolution of halite (NaCl) over time in pure water. In this tutorial, we'll instead use a reaction rate model already available in Reaktoro based on the model of {cite:t}`Palandri2004`:

$$
    r_m=\mathrm{SA}\sum_{i}\left\{ k_{i}^{\circ}\exp\left[-\dfrac{E_{i}}{R}\left(\dfrac{1}{T}-\dfrac{1}{298.15}\right)\right]\prod_{j}a_{i,j}^{w_{i,j}}(1-\Omega^{p_{i}})^{q_{i}}\right\}
$$ (eq:palandri-kharaka-model)


The rate model above computes the rate of dissolution (or precipitation) of a mineral reaction in mol/s:

$$
    \text{Mineral}\rightleftharpoons\text{Aqueous Product Species}.
$$

```{note}
Reaktoro adopts the following sign convention for reaction rates: positive when the reaction proceeds from left to right (i.e., reactants forming products), negative otherwise. Thus, for a mineral reaction, with the mineral considered as a reactant (which is the convention in Reaktoro when the mineral reaction is defined with a mineral name only, without specifying the product species - see next), the reaction rate is *positive when the reaction is dissolving* and *negative when precipitating*.
```

$\mathrm{SA}$ is the current surface area of the mineral in m<sup>2</sup> and $\Omega$ is its current *saturation ratio*:

$$
    \Omega=\dfrac{\mathrm{IAP}}{K}=\dfrac{1}{K}\prod_{i}a^{\nu_{i}} 
$$ (eq:omega)

with $K$ being the equilibrium constant of the mineral reaction and $\nu_i$ the stoichiometric coefficient of the participating $i$th aqueous species. For example, for the mineral reaction:

$$\underset{_{\mathrm{Dolomite}}}{\mathrm{CaMg(CO_{3})_{2}}}=\mathrm{Ca}^{+2}+\mathrm{Mg}^{+2}+2\mathrm{CO_{3}^{2-}}$$

$\nu_i=1$ for both $\mathrm{Ca}^{+2}$ and $\mathrm{Mg}^{+2}$; $\nu_i=2$ for $\mathrm{CO_{3}^{2-}}$; and $\nu_i=0$ for all other species. 

```{note}
Reaktoro calculates the mineral saturation ratio, $\Omega$, using *dual chemical potentials of elements and charge* determined from the current chemical potentials of the aqueous species (see {cite:t}`Smith1982`, {cite:t}`Kulik2006 ` and {cite:t}`Leal2017a`). As long as the mineral reactions proceed under the assumption that all aqueous species are in equilibrium at all times (i.e., a partial equilibrium assumption in which the aqueous species instantaneously reequilibrate as soon as the mineral reacts slightly), then the mineral saturation ratio calculated using this methodology is identical to using equation {eq}`eq:omega`. If this assumption is not applicable, equation {eq}`eq:omega` should be used. In geochemical modeling, this partial equilibrium assumption is generally considered, since aqueous species react at much faster rates than that of mineral dissolution and precipitation.
```

The Palandri-Kharaka rate model {eq}`eq:palandri-kharaka-model` lumps different *reaction mechanisms* that control the reactivity of the mineral and this is reflected by the sum term on the right-hand side. Each reaction mechanism depend on the following parameters:

* $k_{i}^{\circ}$, a reaction rate constant parameter (in mol/(m<sup>2</sup>{{cdot}}s));
* $E_{i}$, an activation energy parameter (in J/mol);
* $w_{i,j}$, an exponent parameter on the activity of species $j$ (dimensionless);
* $p_{i}$, an exponent parameter on the saturation ratio $\Omega$ of the mineral (dimensionless);
* $q_{i}$, an exponent parameter on the term $(1 - \Omega)^{p_i}$ that measures disequilibrium (dimensionless).

For mineral gibbsite (Al(OH)<sub>3</sub>), the Palandri-Kharaka rate model contains three reaction mechanisms: *acid*, *neutral*, and *base*, and it depends on the activity of species H<sup>+</sup>. Below are the values of the above parameters for each mechanism, taken from {cite:t}`Palandri2004`:


```{table}
:name: gibbsite-params-table
:align: center

| Mechanism | $\log k^\circ$ [log mol/(m<sup>2</sup>{{cdot}}s)] | $E$ [kJ/mol] | $w_{\mathrm{H^+}}$ |
|-----------|---------------------------------------------------|--------------|--------------------|
| acid      | -7.65                                             | 47.5         | 0.992              |
| neutral   | -11.50                                            | 61.2         | 0.0                |
| base      | -16.65                                            | 80.1         | -0.784             |
```

Let's now take advantage of this model to simulate the dissolution of carbonate minerals.

In [9]:
from reaktoro import *

db = SupcrtDatabase("supcrtbl")

In [10]:
params = Params.embedded("PalandriKharaka.yaml")

In [11]:
# print(params.data())

In [24]:
system = ChemicalSystem(db,
    AqueousPhase().set(chain(ActivityModelDavies(), ActivityModelDrummond("CO2"))), # combine Davies model for ions with Drummond (1981) model for CO2(aq)
    GaseousPhase("CO2(g)").set(ActivityModelPengRobinson()),
    MineralPhases("Calcite Magnesite Dolomite"),
    MineralReaction("Calcite").setRateModel(ReactionRateModelPalandriKharaka(params)),
    MineralReaction("Magnesite").setRateModel(ReactionRateModelPalandriKharaka(params)),
    MineralReaction("Dolomite").setRateModel(ReactionRateModelPalandriKharaka(params)),
    MineralSurface("Calcite", 6.0, "cm2/cm3"),
    MineralSurface("Magnesite", 6.0, "cm2/cm3"),
    MineralSurface("Dolomite", 6.0, "cm2/cm3"),
)

In [39]:
state = ChemicalState(system)
state.set("H2O(aq)"  , 1.0, "kg")
state.set("CO2(g)"   , 5.0, "mol")
state.set("Calcite"  , 1.0, "mol")
state.set("Magnesite", 1.0, "mol")
state.set("O2(aq)"   , 1.0, "umol")  # Important: when O2(aq) or H2(aq) are in the system, add an irrelevant amount of one of them to avoid numerical problems due to floating-point rounding errors.

solver = KineticsSolver(system)  # the chemical kinetics solver

# Auxiliary constants
minute = 60
hour   = 60 * minute
day    = 24 * hour

duration = 1500 * day  # simulate kinetics for 1500 days
steps = 500  # consider 500 time steps

dt = duration/steps  # compute time step

# Initiate the chemical kinetics simulation
table = Table()  # used to create a table of collected data during the simulation

for i in range(steps + 1):

    result = solver.solve(state, dt)  # compute the chemical state of the system after it reacted for given time length

    assert result.succeeded(), f"Calculation did not succeed at time step #{i}."

    props = state.props()  # get the current thermodynamic and chemical properties of the system
    aprops = AqueousProps.compute(props)  # compute the current aqueous properties of the system

    table.column("Time")           << i*dt / day  # convert time from seconds to days
    table.column("Calcite")        << props.speciesAmount("Calcite")   # in mol
    table.column("Magnesite")      << props.speciesAmount("Magnesite") # in mol
    table.column("Dolomite")       << props.speciesAmount("Dolomite")  # in mol
    table.column("Ca+2")           << props.speciesAmount("Ca+2")      # in mol
    table.column("Mg+2")           << props.speciesAmount("Mg+2")      # in mol
    table.column("RateCalcite")    << props.reactionRate("Calcite")    # in mol/s
    table.column("RateMagnesite")  << props.reactionRate("Magnesite")  # in mol/s
    table.column("RateDolomite")   << props.reactionRate("Dolomite")   # in mol/s
    table.column("pH")             << aprops.pH()
    table.column("OmegaCalcite")   << aprops.saturationRatio("Calcite")
    table.column("OmegaMagnesite") << aprops.saturationRatio("Magnesite")
    table.column("OmegaDolomite")  << aprops.saturationRatio("Dolomite")

In [40]:
table.save("data.txt")

In [41]:
from reaktplot import *

fig1 = Figure()
fig1.title("AQUEOUS SPECIES AMOUNTS OVER TIME")
fig1.xaxisTitle("Time [day]")
fig1.yaxisTitle("Amount [mol]")
fig1.drawLine(table["Time"], table["Ca+2"], "Ca<sup>2+</sup>")
fig1.drawLine(table["Time"], table["Mg+2"], "Mg<sup>2+</sup>")
fig1.show()

In [42]:
fig2 = Figure()
fig2.title("MINERAL AMOUNTS OVER TIME")
fig2.xaxisTitle("Time [day]")
fig2.yaxisTitle("Amount [mol]")
fig2.drawLine(table["Time"], table["Calcite"], "Calcite")
fig2.drawLine(table["Time"], table["Magnesite"], "Magnesite")
fig2.drawLine(table["Time"], table["Dolomite"], "Dolomite")
fig2.show()

In [43]:
fig3 = Figure()
fig3.title("PH OVER TIME")
fig3.xaxisTitle("Time [day]")
fig3.yaxisTitle("pH")
fig3.drawLine(table["Time"], table["pH"], "pH")
fig3.show()

In [44]:
fig4 = Figure()
fig4.title("REACTION RATES OVER TIME")
fig4.xaxisTitle("Time [day]")
fig4.yaxisTitle("Rate [mol/s]")
fig4.drawLine(table["Time"], table["RateCalcite"], "Calcite")
fig4.drawLine(table["Time"], table["RateMagnesite"], "Magnesite")
fig4.drawLine(table["Time"], table["RateDolomite"], "Dolomite")
fig4.show()